In [2]:
getwd()

In [ ]:
setwd('C:/Users/rssch/Desktop/classes/Data Analysis Class 2019/Module 1'

In [ ]:
tbrData <- read.delim("TbrMappedCountMatrix.txt",row.names="Gene"

In [ ]:
head(tbrData)

In [ ]:
head(tbrData,25)

In [ ]:
summary(tbrData)

In [ ]:
dim(tbrData)

In [ ]:
tbrData <- tbrData[substr(row.names(tbrData),1,3)=="PMI",];
dim(tbrData)

In [ ]:
tbrDataLogNorm <- tbrData
for (i in 1:dim(tbrData)[1]) 
{
 tbrDataLogNorm[i,] <- log((as.numeric(tbrData[i,])+0.1)/mean(as.numeric(tbrData[i,])+0.1))
}

In [ ]:
dim(tbrDataLogNorm)
dim(tbrData)

In [ ]:
summary(tbrDataLogNorm)

In [ ]:
tbrDataLogNorm["PMI_005206",]
t.test(x=tbrDataLogNorm["PMI_005206",c("Control1","Control2","Control3")],
       y=tbrDataLogNorm["PMI_005206",c("Tbr1","Tbr2","Tbr3")],var.equal=TRUE)$p.value

In [ ]:
pvalue <- 0.01
significantGenes <- c()
for (i in 1:dim(tbrData)[1])
{
 if (sum(tbrData[i,c("Control1","Control2","Control3")])>10 &&   
    sum(tbrData[i,c("Tbr1","Tbr2","Tbr3")])>10)
 {
  if (t.test(x=tbrDataLogNorm[i,c("Control1","Control2","Control3")],
              y=tbrDataLogNorm[i,c("Tbr1","Tbr2","Tbr3")],
              var.equal=TRUE)$p.value < pvalue)
  {significantGenes <-  append(significantGenes,rownames(tbrDataLogNorm[i,]))}
 }
}

In [ ]:
length(significantGenes)

In [ ]:
head(significantGenes)

In [ ]:
t.test(x=tbrDataLogNorm["PMI_001926",c("Control1","Control2","Control3")],y=tbrDataLogNorm["PMI_001926",c("Tbr1","Tbr2","Tbr3")],var.equal=TRUE)$p.value

In [ ]:
pvalue <- 0.01
significantGenes <- c()
for (i in 1:dim(tbrData)[1])
{
 if (sum(tbrData[i,c("Control1","Control2","Control3")])>10 &&   
    sum(tbrData[i,c("Tbr1","Tbr2","Tbr3")])>10)
 {
  if(t.test(x=tbrDataLogNorm[i,c("Control1","Control2","Control3")],
     y=tbrDataLogNorm[i,c("Tbr1","Tbr2","Tbr3")],
     var.equal=TRUE)$p.value < pvalue/dim(tbrDataLogNorm)[1])
 {
  significantGenes <- 
   append(significantGenes,rownames(tbrDataLogNorm[i,]))}
 }
}
 
length(significantGenes)

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()

BiocManager::install(c("GenomicFeatures", "AnnotationDbi"))
BiocManager::install(c("edgeR"))a
BiocManager::install(c("DESeq"))

In [ ]:
library("edgeR")

In [ ]:
tbrData <- read.delim("TbrMappedCountMatrix.txt",row.names="Gene")

In [ ]:
tbrDataTrimmed <- subset(tbrData,(Control1>5)+(Control2>5)+(Control3>5)+(Tbr1>5)+(Tbr2>5)+(Tbr3>5)>2)

In [ ]:
dim(tbrDataTrimmed)

In [ ]:
head(tbrDataTrimmed,25)

In [ ]:
summary(tbrDataTrimmed)

In [ ]:
group <- factor(c(1,1,1,2,2,2))
design <- model.matrix(~group)

In [ ]:
dge <- DGEList(counts=tbrDataTrimmed,group=group)

In [ ]:
dge <- calcNormFactors(dge)

In [ ]:
dge <- estimateDisp(dge,design)

In [ ]:
fit <- glmFit(dge,design)

In [ ]:
lrt <- glmLRT(fit,coef=2)
lrt

In [ ]:
lrt["PMI_005206",]$table

In [ ]:
t.test(x=tbrDataLogNorm["PMI_005206",c("Control1","Control2","Control3")],y=tbrDataLogNorm["PMI_005206",c("Tbr1","Tbr2","Tbr3")],var.equal=TRUE)$p.value

In [ ]:
topTags(lrt)

In [ ]:
t.test(x=tbrDataLogNorm["PMI_007921",c("Control1","Control2","Control3")],y=tbrDataLogNorm["PMI_007921",c("Tbr1","Tbr2","Tbr3")],var.equal=TRUE)$p.value

In [ ]:
tbrDataTrimmed["PMI_007921",]
tbrDataTrimmed["PMI_027270",]
tbrDataTrimmed["PMI_011495",]

In [ ]:
edgeResults <- topTags(lrt,n=10000,p.value=0.01/dim(tbrData)[1])

In [ ]:
dim(edgeResults)

In [ ]:
write.table(rownames(edgeResults),"significantTbrGenes.txt")
write.table(rownames(edgeResults),"significantTbrGeneNames.txt",col.names=FALSE,row.names=FALSE,quote=FALSE)